In [1]:
%load_ext autoreload

import os

import pandas as pd

print( os.getcwd() )
# change working directory
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util as du

/var/genie-in-the-box/src/ephemera/notebooks
/var/genie-in-the-box/src


In [3]:
# associate commands with files containing examples
browser_commands = {
    "search new tab": "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt",
    "search current tab": "/src/ephemera/prompts/data/synthetic-data-search-in-current-tab.txt",
    "search google new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-new-tab.txt",
    "search google current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-current-tab.txt",
    "search google scholar new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-new-tab.txt",
    "search google scholar current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-current-tab.txt",
}

In [4]:
# prefix = "/var/model/genie-in-the-box"
prefix = du.get_project_root()

for command in browser_commands.keys():
    print( os.path.exists( prefix + browser_commands[ command ] ) )

True
True
True
True
True
True


In [5]:
raw_lines = du.get_file_as_list( prefix + "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt", clean=True, randomize=True )[ 0:100 ]
raw_lines[ 0:10 ]

['Can we search SEARCH_TERMS in a new tab?',
 'Find SEARCH_TERMS, open in new tab',
 'Do a surge, display the results in a gnu tab',
 'Kindly start SEARCH_TERMS search in another tab',
 'Zoo a search and display the results in a new dab',
 'Joe a surge, and display the rustles in a new tab',
 'Joe a merch and dismay the rustles in a new dab',
 'Throw a birch and dis play, the rustles in another tub',
 'Execute new tab, search for SEARCH_TERMS',
 'Can you search for SEARCH_TERMS in a new tab?']

In [6]:
def get_search_terms( requested_length ):
    
    # Load search terms file
    search_terms = du.get_file_as_list( prefix + "/src/ephemera/prompts/data/search-terms.txt", lower_case=False, clean=True, randomize=True )
    
    # If we don't have enough search terms, append copies of the search term list until we do
    while requested_length > len( search_terms ):
        search_terms += search_terms
        
    # Truncate the search terms list to equal the requested len
    search_terms = search_terms[ :requested_length ]
    
    return search_terms

In [7]:
search_terms = get_search_terms( len( raw_lines ) )
search_terms[ 0:10]

['Quantum AI computing developments: What are the latest developments in quantum AI computing?',
 'AI in virtual reality: How is AI enhancing virtual reality experiences?',
 'Index Error: Index out of bounds',
 'DeprecationWarning',
 'best burger joints',
 'Merging strategies in Pandas',
 'what is climate change?',
 'Reference Error: Weak reference object no longer exists',
 'Machine Learning in biotech',
 'User Warning']

In [8]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
command_choices 

"'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'"

In [71]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
instruction_template = """Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command."""

input_template = """
Below is the raw human voice command transcription formatted using simple XML: 
{human_says}

The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}"""

human_says_template = """
<human>
    <voice-command>{voice_command}</voice-command>
</human>"""

response_format = """
<response>
    <browser-command></browser-command>
    <args></args>
</response>

Requirement: The first word of your response MUST be `<response>`"""

output_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>"""

In [72]:
for line in instruction_template.split( "\n" ):
    print( line )

Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.


In [73]:
instruction = instruction_template.format( command_choices=command_choices)
instruction

"Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.\n\nRequirement: You MUST NOT use python code to answer this question.\nRequirement: You MUST use your linguistic knowledge and intuition to answer this question.\nHint: Anything that isn't a part of the command itself should be treated as arguments related to the command."

In [77]:
def prompt_instruction_format( instruction, input ):

    return f"""### Instruction:
Use the Task and Input given below to write a Response that can solve the following Task.

### Task:
{instruction}

### Input:{input}
### Response:
"""

In [78]:
instructions = []
inputs       = []
outputs      = []
prompts      = []

for browser_command in browser_commands.keys():
    
    du.print_banner( browser_command, prepend_nl=True, end="\n" )
    
    raw_lines = du.get_file_as_list( prefix + browser_commands[ browser_command ], clean=True )
    for line in raw_lines[ 0:100 ]: #[ 0:2 ]:#
        
        # get newly randomized search terms on every iteration
        for terms in get_search_terms( len( raw_lines ) ):#[ 0:10 ]: #[ 0:2]:#:
            
            voice_command = line.replace( "SEARCH_TERMS", terms )
            instruction   = instruction_template.format( command_choices=command_choices )
            human_says    = human_says_template.format( voice_command=voice_command )
            input         = input_template.format( human_says=human_says, response_format=response_format )
            output        = output_template.format( browser_command=browser_command, args=terms )
            prompt        = prompt_instruction_format( instruction, input )
    
            instructions.append( instruction )
            inputs.append( input )
            outputs.append( output )
            prompts.append( prompt )
    
            print( ".", end="" )
        
    print()


------------------------------------------------------------------------------------------------------------------------
- search new tab
------------------------------------------------------------------------------------------------------------------------
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [79]:
for line in prompts[ 0 ].split( "\n" ):
    print( line )

### Instruction:
Use the Task and Input given below to write a Response that can solve the following Task.

### Task:
Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

### Input:
Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <v

In [80]:
idx = 0

for line in instructions[ 0 ].split( "\n" ):
    print( line )
    
for line in inputs[ 0 ].split( "\n" ):
    print( line )
    
for line in outputs[ idx ].split( "\n" ):
    print( line )

Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Requirement: You MUST NOT use python code to answer this question.
Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <voice-command>Creating scatter plots with Pandas new tab search</voice-command>
</human>

The standardized command that you transl

In [19]:
import random

def inject_space( line ):
    if random.random() < 0.5:
        index = random.randint( 0, len( line ) - 1 )
        return line[ :index ] + ' ' + line[ index: ]
    else:
        return line

In [81]:
qna_df = pd.DataFrame( { "instruction": instructions, "input": inputs, "output": outputs, "prompt": prompts } )

# Perturb the answer to create a faked response  
# qna_df[ "response" ] = qna_df[ "output" ].apply( lambda cell: inject_space( cell ) )
qna_df.head()

,instruction,input,output,prompt
0,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...
1,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...
2,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...
3,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...
4,Your job is to discern the intent of a human v...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\nUse the Task and Input given...


## Validate the structure of the xml response

In [22]:
!pip install xmlschema


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from xmlschema import XMLSchema

xml_str_1 = """
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""
xml_str_2 = """
<response>
  <browser_command></browser_command>
  <args></args>
</response>
"""
xml_str_3 = """
Sure here's your command!
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xml_str_4 = """
```xml
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xsd_string = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="response">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="browser-command" type="xs:string"/>
        <xs:element name="args" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

def is_valid_xml( xml_str, schema ):
    
    try:
        schema.is_valid( xml_str )
        return True
    except Exception as e:
        # print( "Invalid XML: " + str( e ) )
        return False

schema   = XMLSchema( xsd_string )

print( is_valid_xml( xml_str_1, schema ) )  # Output: ???
print( is_valid_xml( xml_str_2, schema ) )  # Output: ???
print( is_valid_xml( xml_str_3, schema ) )  # Output: False
print( is_valid_xml( xml_str_4, schema ) )  # Output: False

True
True
False
False


In [24]:
qna_df[ "response_xml_is_valid" ] = qna_df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
qna_df.head()

,instruction,input,output,prompt,response,response_xml_is_valid
0,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True
1,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<resp onse>\n <browser-command>search new...,False
2,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search ne w...,True
3,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,False
4,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True


In [25]:
qna_df.response_xml_is_valid.mean()

0.7953666666666667

In [26]:
def contains_valid_xml_tag( xml_str, tag_name ):
    
    try:
        return "<" + tag_name + ">" in xml_str and "</" + tag_name + ">" in xml_str
    except Exception as e:
        return False
    
contains_valid_xml_tag( "```xml<response><browser-command></browser-command><args></args></response>", "browser-command")

True

## Validate the structure of the xml response

In [27]:
qna_df[ "contains_response" ]        = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
qna_df[ "contains_browser_command" ] = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
qna_df[ "contains_args" ]            = qna_df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )

In [28]:
print( qna_df[ "contains_response" ].mean() )
print( qna_df[ "contains_browser_command" ].mean() )
print( qna_df[ "contains_args" ].mean() )

0.9319416666666667
0.881125
0.9603333333333334


## Validate the response by removing all white space and comparing to the original answer

In [29]:
import re

xml_str = """
<response>
  <browser-command>Whos your favorite browser?</browser-command>
  <args>Bar browser</args>
</response>
"""

# Remove white space outside XML tags
xml_str = re.sub(r'>\s+<', '><', xml_str.strip())

print(xml_str)

<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>


In [30]:
def is_response_exact( response, answer ):
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

is_response_exact( "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [40]:
def tag_values_are_equal( response, answer, tag_name="browser-command" ):
    
    command_response = dux.get_value_by_xml_tag_name( response, tag_name, default_value="broken" )
    command_answer   = dux.get_value_by_xml_tag_name(   answer, tag_name, default_value="broken" )
    
    return command_response != "broken" and command_answer != "broken" and command_response == command_answer

response = "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>"
answer   = "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>"

print( tag_values_are_equal( response, answer, tag_name="browser-command" ) )
print( tag_values_are_equal( response, answer, tag_name="args" ) )

True
True


In [31]:
import lib.utils.util_xml as dux

def contains_correct_response_values( response, answer ):
    
    """Check to see if the most common formatting error (```xml) is hiding a correct response"""
    response = "<response>" + dux.get_value_by_xml_tag_name( response, "response", default_value="broken" ) + "</response>"
    response = dux.get_xml_tag_and_value_by_name( response, "response", default_value="broken" )
    if response == "broken":
        return False
    
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

contains_correct_response_values( "```xml<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [79]:
qna_df[ "response_is_correct" ] = qna_df.apply( lambda row: is_response_exact( row[ "response" ], row[ "output" ] ), axis=1 )

In [80]:
qna_df.response_is_correct.mean()

0.5608916666666667

In [81]:
qna_df[ ( qna_df.response_xml_is_valid == True ) & ( qna_df.response_is_correct == False ) ].head( 10 )

,instruction,input,output,prompt,response,response_xml_is_valid,contains_response,contains_browser_command,contains_args,response_is_correct
1,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
2,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
5,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
6,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command> search new...,True,True,True,True,False
9,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
18,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command >search new...,True,True,False,True,False
19,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
20,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,False,True,False
29,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False
39,\nYour job is to discern the intent of a human...,\nBelow is the raw human voice command transcr...,\n<response>\n <browser-command>search new ...,### Instruction:\n Use the Task and Input g...,\n<response>\n <browser-command>search new ...,True,True,True,True,False


# What's the baseline behavior of the Vanilla model?

In [82]:
# !pip install openai
!pip install tiktoken


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [83]:
%autoreload
from huggingface_hub import InferenceClient
from lib.utils.util_stopwatch import Stopwatch
from lib.agents.agent import Agent

def query_llm_phind( prompt, model=Agent.PHIND_34B_v2, max_new_tokens=1024, temperature=0.25, top_k=10, top_p=0.9, debug=False, verbose=False, silent=True ):
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ), silent=silent )
    
    # Get the TGI server URL for this context
    # default_url    = self.config_mgr.get( "tgi_server_url_phind", default=None )
    # tgi_server_url = du.get_tgi_server_url_for_this_context( default_url=default_url )
    
    client         = InferenceClient( "http://172.17.0.5:3000" )
    token_list     = [ ]
    ellipsis_count = 0
    
    if debug:
        for line in prompt.split( "\n" ):
            print( line )
    
    for token in client.text_generation(
        prompt, max_new_tokens=max_new_tokens, stream=True, temperature=temperature, top_k=top_k, top_p=top_p, stop_sequences=[ "</response>" ]
    ):
        if debug:
            print( token, end="" )
        else:
            if not silent: print( ".", end="" )
            ellipsis_count += 1
            if ellipsis_count == 120:
                ellipsis_count = 0
                print()
            
        token_list.append( token )
        
    response = "".join( token_list ).strip()
    
    timer.print( msg="Done!", use_millis=True, prepend_nl=True, end="\n" )
    tokens_per_second = len( token_list ) / ( timer.get_delta_ms() / 1000.0 )
    print( f"Tokens per second [{round( tokens_per_second, 1 )}]" )
            
    if debug:
        print( f"Token list length [{len( token_list )}]" )
        if verbose:
            for line in response.split( "\n" ):
                print( line )
    
    
    return response

In [35]:
import openai

def query_llm_openai( preamble, query, model=Agent.GPT_3_5, debug=False, verbose=False ):
    
    openai.api_key = du.get_api_key( "openai" )
    
    timer = Stopwatch( msg=f"Asking LLM [{model}]...".format( model ) )
    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            { "role": "system", "content": preamble },
            { "role": "user", "content": query }
        ],
        temperature=0,
        max_tokens=2000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    timer.print( "Done!", use_millis=True )
    if debug and verbose:
        # print( json.dumps( response.to_dict(), indent=4 ) )
        print( response )
    
    return response.choices[ 0 ].message.content.strip()

In [53]:
!pip show openai

Name: openai
Version: 1.5.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [84]:
call_counter = 0
# rows         = df.shape[ 0 ]

def get_response_to_question( prompt, rows ):
    
    global call_counter
    
    call_counter += 1
    print( f"On call [{call_counter:03d}] out of [{rows}] = [{round( call_counter / rows * 100.0, 1 )}%]... ", end="" )
    
    return query_llm_phind( prompt )

In [85]:
def validate( df ):
    
    rows = df.shape[ 0 ]
    global call_counter
    call_counter = 0
    
    df[ "response" ]                    = df[ "prompt" ].apply( lambda cell: get_response_to_question( cell, rows ) )
    
    # Validate the structure and content of the xml response
    df[ "response_xml_is_valid" ]       = df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
    df[ "contains_response" ]           = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "response" ) )
    df[ "contains_browser_command" ]    = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "browser-command" ) )
    df[ "contains_args" ]               = df[ "response" ].apply( lambda cell: contains_valid_xml_tag( cell, "args" ) )
    df[ "response_is_exact" ]           = df.apply( lambda row: is_response_exact( row[ "response" ], row[ "answer" ] ), axis=1 )
    df[ "response_has_correct_values" ] = df.apply( lambda row: contains_correct_response_values( row[ "response" ], row[ "answer" ] ), axis=1 )
    
    return df


In [86]:
def print_stats( df ):

    du.print_banner( "Stats" )
    print( f"               Is valid xml {df.response_xml_is_valid.mean() * 100:.1f}%" )
    print( f"          Contains response {df.contains_response.mean() * 100:.1f}%" )
    print( f"   Contains browser command {df.contains_browser_command.mean() * 100:.1f}%" )
    print( f"              Contains args {df.contains_args.mean() * 100:.1f}%" )
    print( f"          Response is exact {df.response_is_exact.mean() * 100:.1f}%" )
    print( f"Response has correct values {df.response_has_correct_values.mean() * 100:.1f}%" )
    

In [43]:
! pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 89.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 81.8 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [88]:
prompts[ 0 ]

"### Instruction:\n    Use the Task and Input given below to write a Response that can solve the following Task:\n\n    ### Task:\n    \nYour job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.\n\nYou will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.\n\nRequirement: You MUST NOT use python code to answer this question.\nRequirement: You MUST use your linguistic knowledge and intuition to answer this question.\nHint: Anything that isn't a part of the command itself should be treated as arguments related to the command.\n\n\n    ### Input:\n    \nBelow is the raw human voice command transcription form

In [41]:
qna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   instruction               120000 non-null  object
 1   input                     120000 non-null  object
 2   output                    120000 non-null  object
 3   prompt                    120000 non-null  object
 4   response                  120000 non-null  object
 5   response_xml_is_valid     120000 non-null  bool  
 6   contains_response         120000 non-null  bool  
 7   contains_browser_command  120000 non-null  bool  
 8   contains_args             120000 non-null  bool  
dtypes: bool(4), object(5)
memory usage: 5.0+ MB


In [82]:
from sklearn.model_selection import train_test_split

sampled_df = qna_df[ [ "instruction", "input", "output", "prompt" ] ].sample( 1000, random_state=42 ).copy()

# Split the dataframe into train and (test+validate)
train_df, test_validate_df = train_test_split( sampled_df, test_size=0.2, random_state=42 )

# Then split (test+validate) into test and validate
test_df, validate_df = train_test_split( test_validate_df, test_size=0.5, random_state=42 )


In [83]:
print( train_df.shape )
print( test_df.shape )
print( validate_df.shape )

(800, 4)
(100, 4)
(100, 4)


In [91]:
validate_df = validate( validate_df )

print_stats( validate_df )

# First run before attempting to force valid XML
# ------------------------------------------------------------------------------------------------------------------------
# - Stats
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 98.0%
#    Contains browser command 98.0%
#               Contains args 98.0%
#           Response is exact 0.0%
# Response has correct values 43.0%

# after attempting to force the first token output: <response>
# ------------------------------------------------------------------------------------------------------------------------
# - Stats
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 0.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 0.0%
# Response has correct values 39.0%

On call [001] out of [100] = [1.0%]... 

ConnectionError: (MaxRetryError("HTTPConnectionPool(host='172.17.0.5', port=3000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fcfd9108dc0>: Failed to establish a new connection: [Errno 113] No route to host'))"), '(Request ID: 1b75fe79-d094-4be0-bd3b-945bc23ed27e)')

In [191]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 61754 to 88409
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  800 non-null    object
 1   answer    800 non-null    object
dtypes: object(2)
memory usage: 18.8+ KB


In [84]:
sampled_df = qna_df[ [ "instruction", "input", "output", "prompt" ] ].sample( 1000, random_state=42 ).copy()

# Split the dataframe into 80% train and (test+validate)
train_df, test_validate_df = train_test_split( sampled_df, test_size=0.2, random_state=42 )

# Then split (test+validate) into 10% test and 10validate
test_df, validate_df = train_test_split( test_validate_df, test_size=0.5, random_state=42 )

In [86]:
path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-train.jsonl"
train_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-validate.jsonl"
validate_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )

path = du.get_project_root() + "/src/ephemera/prompts/data/search-xml-test.jsonl"
test_df.to_json( path, orient="records", lines=True )
os.chmod( path, 0o666 )